# Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv
import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
## Tags: #Tables
____
## Links:

[[all_installetion_points_parquet]]
[[companies_st_partner]]
[[companies_dir_partner]]
[[billing_orders_devices_st_partner]]
[[cameras_st_partner]]
[[cameras_dir_partner]]
[[intercoms_dir_asgard]]

____
## Creating a table

In [109]:
query_text = """--sql
    CREATE TABLE db1.t_device_billins
    (
        `report_date` Date,
        `service_partner_uuid` String,
        `partner_uuid` String,
        `device_uuid` String,
        `device_serial_number` String,
        `device_type` LowCardinality(String),
        `installation_point_id` UInt32,
        `camera_dvr_depth` UInt8,
        `foreign_camera` UInt8,
        `archive_from_partner` LowCardinality(String),
        `included_by` String,
        `included_at` DateTime,
        `disabled_by` String,
        `disabled_at` DateTime,
        `service` LowCardinality(String),
        `total` Float32,
        `is_blocked` UInt8,
        `pro_subs` UInt8,
        `enterprise_subs` UInt8,
        `billing_pro` UInt8,
        `enterprise_not_paid` UInt8,
        `enterprise_test` UInt8,
        `balance` Float64,
        `kz_pro`UInt8,
        `company_name` String,
        `partner_lk` LowCardinality(String),
        `registration_date` String,
        `tin` String,
        `kpp` String,
        `city` String,
        `country` LowCardinality(String),
        `full_address` String,
        `parent_uuid` String,
        `region` LowCardinality(String)
    )

    ENGINE = MergeTree()
    ORDER BY report_date
    """

ch.query_run(query_text)

""


____
## Creating a MV

In [103]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_device_billins_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 25 MINUTE TO db1.t_device_billins AS
    WITH all_installetion_points_parquet AS (
    SELECT
        `address_uuid`,
        `city`,
        `country`,
        `created_at`,
        `full_address`,
        `installation_point_id`,
        `parent_uuid`,
        `region`,
        `report_date`,
    FROM db1.`all_installetion_points_parquet_ch`
    ),
    companies_st_partner AS (
    SELECT
        `report_date`,
        companies_st_partner.`partner_uuid` as partner_uuid,
        `is_blocked`,
        `pro_subs`,
        `enterprise_subs`,
        `billing_pro`,
        `enterprise_not_paid`,
        `enterprise_test`,
        `balance`,
        `kz_pro`,
        `company_name`,
        `partner_lk`,
        `registration_date`,
        `tin`,
        `kpp`
    FROM db1.`companies_st_partner_ch` AS companies_st_partner 
    LEFT JOIN db1.`companies_dir_partner_ch` AS companies_dir_partner 
        ON  companies_st_partner.partner_uuid = companies_dir_partner.partner_uuid 
    ),
    billing_orders_devices_dir_partner AS (
    SELECT
        `billing_account_id`,
        `cost`,
        `count`,
        toDate(`created_at`) as report_date,
        `device_type`,
        `device_uuid`,
        `partner_uuid`,
        `service`,
        `state`,
        `total`
    FROM db1.`billing_orders_devices_st_partner_ch`
    ),
    cameras_st_partner AS (
    SELECT
        `report_date`,
        cameras_st_partner.`service_partner_uuid` AS service_partner_uuid,
        cameras_st_partner.`partner_uuid` AS partner_uuid,
        if(`motherboard_id`!= '',`motherboard_id`,`serial_number`) AS device_serial_number,
        if(cameras_st_partner.`intercom_uuid` != '',cameras_st_partner.`intercom_uuid` ,cameras_st_partner.`camera_uuid`) AS device_uuid,
        if(cameras_st_partner.`intercom_uuid` != '', 'intercom', if(`foreign_camera` = 1,'camera_foreign','camera')) AS device_type,
        cameras_st_partner.`installation_point_id` AS installation_point_id,
        `camera_dvr_depth`,
        `archive_from_partner`,
        `included_by`,
        `included_at`,
        `disabled_by`,
        `disabled_at`,
        `foreign_camera`
    FROM db1.`cameras_st_partner_ch` AS cameras_st_partner
    LEFT JOIN db1.`cameras_dir_partner_ch` AS cameras_dir_partner 
        ON cameras_st_partner.camera_uuid = cameras_dir_partner.camera_uuid
    LEFT JOIN db1.`intercoms_dir_asgard_ch` AS intercoms_dir_asgard 
        ON cameras_st_partner.intercom_uuid = intercoms_dir_asgard.intercom_uuid
    )
    --
    SELECT 
        DISTINCT
        cameras_st_partner.`report_date` AS report_date,
        `service_partner_uuid`,
        cameras_st_partner.`partner_uuid` AS partner_uuid,
        cameras_st_partner.`device_uuid` AS device_uuid,
        device_serial_number,
        CASE
            WHEN cameras_st_partner.device_type = 'intercom' THEN 'Домофон'
            WHEN cameras_st_partner.device_type = 'camera' THEN 'Камера IZI'
            WHEN cameras_st_partner.device_type = 'camera_foreign' THEN 'Сторонняя камера'
            ELSE NULL
        END AS device_type,
        cameras_st_partner.`installation_point_id` AS installation_point_id,
        `camera_dvr_depth`,
        `foreign_camera`,
        `archive_from_partner`,
        `included_by`,
        `included_at`,
        `disabled_by`,
        `disabled_at`,
        IF(`service`!='', `service`, 'Нет услуг') AS `service`,
        `total`,
        `is_blocked`,
        `pro_subs`,
        `enterprise_subs`,
        `billing_pro`,
        `enterprise_not_paid`,
        `enterprise_test`,
        `balance`,
        `kz_pro`,
        `company_name`,
        `partner_lk` ,
        `registration_date`
        `tin`,
        `kpp`,
        `city`,
        `country`,
        `full_address`,
        `parent_uuid`,
        `region`
    FROM cameras_st_partner
    LEFT JOIN billing_orders_devices_dir_partner
        ON billing_orders_devices_dir_partner.report_date = cameras_st_partner.report_date    
        AND billing_orders_devices_dir_partner.device_uuid = cameras_st_partner.device_uuid
    LEFT JOIN  companies_st_partner
        ON companies_st_partner.report_date = cameras_st_partner.report_date    
        AND companies_st_partner.partner_uuid = cameras_st_partner.partner_uuid
    LEFT JOIN all_installetion_points_parquet
        ON all_installetion_points_parquet.report_date = cameras_st_partner.report_date    
        AND all_installetion_points_parquet.installation_point_id = cameras_st_partner.installation_point_id
    """
ch.query_run(query_text)

""


___
## query

In [2]:
query_text = """ 
SELECT
    *
FROM db1.t_device_billins
WHERE report_date = '2025-06-07'
limit 10
"""

ch.query_run(query_text)

report_date,service_partner_uuid,partner_uuid,device_uuid,device_serial_number,device_type,installation_point_id,camera_dvr_depth,foreign_camera,archive_from_partner,included_by,included_at,disabled_by,disabled_at,service,total,is_blocked,pro_subs,enterprise_subs,billing_pro,enterprise_not_paid,enterprise_test,balance,kz_pro,company_name,partner_lk,registration_date,tin,kpp,city,country,full_address,parent_uuid,region
date,str,str,str,str,str,i64,i64,i64,str,str,datetime[ns],str,datetime[ns],str,f64,i64,i64,i64,i64,i64,i64,f64,i64,str,str,str,str,str,str,str,str,str,str
2025-06-07,"""af8b5bf4-0108-4a0c-acc2-096f94…","""af8b5bf4-0108-4a0c-acc2-096f94…","""884cabd1-0fff-4210-a801-e02c11…","""1835100129""","""Домофон""",940833,0,0,"""""","""""",1970-01-01 00:00:00,"""""",1970-01-01 00:00:00,"""Нет услуг""",0.0,0,1,0,1,0,0,5.10995e6,1,"""ТОО РИТМ ПРО""","""120074""","""""","""2019-04-05""","""""","""Семей""","""Казахстан""","""Казахстан, область Абай, город…","""854a1ab9-951f-40d2-ba0b-bec543…","""область Абай"""
2025-06-07,"""963e8bcc-c6c3-46ee-958f-412c22…","""963e8bcc-c6c3-46ee-958f-412c22…","""221d8b57-d74f-49c8-9422-38b2d5…","""DS-2CDVT-NONE-S0120160606AAWR6…","""Домофон""",44,0,0,"""""","""""",1970-01-01 00:00:00,"""""",1970-01-01 00:00:00,"""Нет услуг""",0.0,0,1,0,1,0,0,2.23045e6,0,"""ООО Bид Плюс ПРО""","""120696""","""""","""2019-11-20""","""741301001""","""Минск""","""Беларусь""","""Беларусь, Минск, Минск, ул. Та…","""d312e75f-8a0b-4404-88f3-0620a0…","""Минск"""
2025-06-07,"""3b98f0a6-6e68-44cc-8c04-7df695…","""3b98f0a6-6e68-44cc-8c04-7df695…","""d6b40231-558b-4d44-ba10-b0b481…","""1809100050""","""Домофон""",164327,0,0,"""""","""""",1970-01-01 00:00:00,"""""",1970-01-01 00:00:00,"""Нет услуг""",0.0,0,0,0,0,0,0,0.0,0,"""Тестовая Спутник""","""155242""","""""","""2024-12-18""","""""","""Йошкар-Ола""","""Россия""","""Россия, Республика Марий Эл, г…","""6bd6e6f3-2d6d-439c-8009-2ccdb8…","""Республика Марий Эл"""
2025-06-07,"""e268f732-1d98-4e27-998a-9a704d…","""e268f732-1d98-4e27-998a-9a704d…","""688fa517-a819-4157-bd79-6d237e…","""1802100009""","""Домофон""",47,0,0,"""""","""""",1970-01-01 00:00:00,"""""",1970-01-01 00:00:00,"""Нет услуг""",0.0,0,0,0,0,0,0,0.0,0,"""1""","""150375""","""""","""2024-03-19""","""""","""Семей""","""Казахстан""","""Казахстан, Восточно-Казахстанс…","""6513828a-979e-4d63-aaba-6cf2f1…","""Восточно-Казахстанская область"""
2025-06-07,"""b1782e4f-9198-49d1-b5aa-7bdba9…","""b1782e4f-9198-49d1-b5aa-7bdba9…","""8657344b-e9ee-48bc-9f2c-45d11a…","""1809100122""","""Домофон""",59,0,0,"""""","""""",1970-01-01 00:00:00,"""""",1970-01-01 00:00:00,"""Нет услуг""",0.0,0,0,1,0,0,0,8.4549e9,0,"""Спутник""","""120012""","""""","""2018-10-30""","""111111111""","""Семей""","""Казахстан""","""Казахстан, Восточно-Казахстанс…","""6513828a-979e-4d63-aaba-6cf2f1…","""Восточно-Казахстанская область"""
2025-06-07,"""db61f873-dcf7-4678-aec1-6b93f5…","""b1782e4f-9198-49d1-b5aa-7bdba9…","""d8d45214-6a8f-4240-a6b7-a8a37f…","""1809100093""","""Домофон""",0,0,0,"""""","""""",1970-01-01 00:00:00,"""""",1970-01-01 00:00:00,"""Нет услуг""",0.0,0,0,1,0,0,0,8.4549e9,0,"""Спутник""","""120012""","""""","""2018-10-30""","""111111111""","""""","""""","""""","""""",""""""
2025-06-07,"""7b5390cf-7640-4f65-8978-faf1cd…","""7b5390cf-7640-4f65-8978-faf1cd…","""df492dfe-8237-4a57-9090-b7175f…","""1843100134""","""Домофон""",68,0,0,"""""","""""",1970-01-01 00:00:00,"""""",1970-01-01 00:00:00,"""Нет услуг""",0.0,0,0,0,0,0,0,0.0,0,"""Спутник Гарантия""","""129297""","""""","""2022-09-13""","""""","""Иркутск""","""Россия""","""Россия, Иркутская область, гор…","""79b38493-91b1-4ccd-965b-817b26…","""Иркутская область"""
2025-06-07,"""""","""c4cea074-f4b6-47ba-af6c-97422b…","""9898dea4-6c63-4339-9201-bf7a27…","""1845100138""","""Домофон""",73,0,0,"""""","""""",1970-01-01 00:00:00,"""""",1970-01-01 00:00:00,"""Нет услуг""",0.0,0,0,1,0,0,0,-1080.0,0,"""MIRCOD""","""120061""","""""","""2019-02-04""","""165501001""","""Нур-Султан (Астана)""","""Казахста

___
## DROP

In [107]:
query_text = """ 
DROP TABLE db1.t_device_billins
"""

ch.query_run(query_text)

,elapsed_ns,query_id
0,697382596,e5ec0a94-84f4-49f5-a72f-9b215c2b95dd


___
## REFRESH

In [112]:
query_text = """
SYSTEM REFRESH VIEW db1.t_device_billins_mv
"""

ch.query_run(query_text)

,elapsed_ns,query_id
0,1128602,c9bf9c09-e880-4719-a4db-3f33ea099448
